We made a test runner that kinda works. Buuut....

- What if we have several different subclasses of Test and we want to run all the tests at once?
- What if we have some code that we need to run before and after every test?
- What if we want to be able to skip a test, or only run specific tests according to some rule we made up?

It is time for us to make these dreams come to life! But first, let's import the matchers we worked on last time so we can keep using them

In [ ]:
from phoenix_test.matchers import FailedAssertion, Assertion, assert_that

Let's also import colorama, because we've made the test runner print in colors, and who doesn't love colors?

In [ ]:
import sys
!{sys.executable} -m pip install colorama 

from colorama import Fore, Back, Style 

Okay, here's where we got to with the test runner last time.

In [ ]:
class Test():
    # Runs all the test methods. HOW?!?!
    def run(self):
        run_count = 0
        pass_count = 0
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            run_count += 1
            try:
                getattr(self.__class__, method).__call__(self)
                pass_count += 1
                print(Fore.GREEN + f"{method} passed!")
            except Exception as e:
                print(Fore.RED +f"{method}:  {e}") 
        print(Style.RESET_ALL)
        print(f"{pass_count} out of {run_count} tests passed.")

And when we run it on some example tests, we get some beautiful test output. Here, give it a try!

In [ ]:
def find_twos(one, two):
    return []

class FindTwosTest(Test):

    def test_empty_inputs(self):
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])
        
FindTwosTest().run()

So this is where we are.

### Challenge:

What if I have several different subclasses of Test and I want to run all my tests at once? So, in addition to my `FindTwosTest`, I also want to run this spiffy test suite:

In [ ]:
class SortedTests(Test):
    def test_sort_integers(self):
        assert_that(sorted([3, 1, 2])).equals([1, 2, 3])

    def test_sort_strings(self):
        assert_that(sorted(["C", "A", "B"])).equals(["A", "B", "C"])

SortedTests().run()

### Challenge: make this statement run all the tests:

`Test.run_all()`

Such that, whichever classes have subclassed `Test`, they all run when you run that command. The output might look something like this:

![](../images/run_all_output.png)

Hint: use `__init_subclass__`. This method is run on a superclass every time a subclass gets instantiated. You can use it to register all of the subclasses in a collection at the superclass level. You can read all about exactly how this method works in [the PEP proposal](https://www.python.org/dev/peps/pep-0487/) that introduced it to Python!

Here is our Test class so far with some starter code added for you:

In [ ]:
class Test():
    types = []
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        # Add the initialized class to the types list here!
    
    @classmethod
    def run_all(cls):
        pass
        # What do we do inside here?

    def run(self):
        run_count = 0
        pass_count = 0
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            run_count += 1
            try:
                getattr(self.__class__, method).__call__(self)
                pass_count += 1
                print(Fore.GREEN + f"    {method} passed!")
            except Exception as e:
                print(Fore.RED +f"    {method}:  {e}") 
        print(Style.RESET_ALL + f"    {pass_count} out of {run_count} tests passed.\n")
        return pass_count, run_count


Here are all our test classes together, so you can easily check whether all of your test methods are getting run!

In [ ]:
class FindTwosTest(Test):

    def test_empty_inputs(self):
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])
        
class SortedTests(Test):
    def test_sort_integers(self):
        assert_that(sorted([3, 1, 2])).equals([1, 2, 3])

    def test_sort_strings(self):
        assert_that(sorted(["C", "A", "B"])).equals(["A", "B", "C"])

Test.run_all()

### Challenge: 

1. Make the test suite print the name of each test class before running it
2. Make the test suite print a total number of tests run and passed

**Something like this:**

![](../images/test_output.png)

In [ ]:
Test.run_all()

### Challenge: 

What if I have some code that I need to run before and after every test? Suppose, for example, that I am keeping a cache:

In [ ]:
class CachedStringQueue:
    items = []
    
    @classmethod
    def validate(cls):
        invalids = 0
        for item in CachedStringQueue.items:
            if item == 'invalid':
                invalids += 1
        cls.items = sorted(cls.items)[invalids:]

And I have two test suites for the cache: one that checks how to put things in it (assuming an empty cache) and one that checks how to invalidate items (which requires some stuff to be in the cache.)

Perhaps we could include `setup` and `teardown` methods in our test class that run before and after each separate test:

In [ ]:
class CacheSetAndFetchTests(Test):
    def test_set_and_fetch(self):
        CachedStringQueue.items.append("stringo")
        assert_that(CachedStringQueue.items.pop(0)).equals("stringo")
    
class CacheValidationTests(Test):
    def setup(self):
        print("SETUP RUN")
        CachedStringQueue.items = ['valid', 'invalid', 'valid']
    
    def teardown(self):
        print("TEARDOWN RUN")
        CachedStringQueue.items = []
        
    def test_validation(self):
        CachedStringQueue.validate()
        assert_that(CachedStringQueue.items).has_size(2)
        assert_that(set(CachedStringQueue.items)).equals({'valid'})

Right now, those functions are never run: 

In [ ]:
CacheSetAndFetchTests().run()
CacheValidationTests().run()

But if they _were_ run at the appropriate times, we'd expect to see some output like this (don't worry about the colors of the prints for now): 

![](../images/setup_teardown.png)

## Challenge:

Add this `setup` and `teardown` function to our test functionality. Remember to call those methods in the test runner to run before and after each test!

In [ ]:
class Test():
    types = []
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.types.append(cls)
    
    @classmethod
    def run_all(cls):
        pass_count = 0
        run_count = 0
        
        for typ in cls.types:
            print(f"Running {typ.__name__}: ")
            passed, runned = typ().run() 
            pass_count += passed
            run_count += runned
        
        print(f"{pass_count} out of {run_count} tests passed.\n")
    
        
    def run(self):
        run_count = 0
        pass_count = 0
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            run_count += 1
            try:
                getattr(self.__class__, method).__call__(self)
                pass_count += 1
                print(Fore.GREEN + f"    {method} passed!")
            except Exception as e:
                print(Fore.RED +f"    {method}:  {e}") 
        print(Style.RESET_ALL + f"    {pass_count} out of {run_count} tests passed.\n")
        return pass_count, run_count


In [ ]:
CacheSetAndFetchTests().run()
CacheValidationTests().run()

### Challenge: 

What if I want to be able to only run specific tests according to some rule I made up?

Maybe:

`Test.run_all(only="match")` only runs tests with "match" in their name. You can modify this version of the test runner to make that happen:

In [ ]:
class Test():
    types = []
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.types.append(cls)
    
    @classmethod
    def run_all(cls):
        pass_count = 0
        run_count = 0
        
        for typ in cls.types:
            print(f"Running {typ.__name__}: ")
            passed, runned = typ().run() 
            pass_count += passed
            run_count += runned
        
        print(f"{pass_count} out of {run_count} tests passed.\n")
    
    def setup(self):
        pass
    
    def teardown(self):
        pass
        
    def run(self):
        run_count = 0
        pass_count = 0
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            self.setup()
            run_count += 1
            try:
                getattr(self.__class__, method).__call__(self)
                pass_count += 1
                print(Fore.GREEN + f"    {method} passed!")
            except Exception as e:
                print(Fore.RED +f"    {method}:  {e}") 
            self.teardown()
        print(Style.RESET_ALL + f"    {pass_count} out of {run_count} tests passed.\n")
        return pass_count, run_count


This time we'll _just_ run `FindTwosTest` to keep our example small while making sure that our code works:

In [ ]:
class FindTwosTest(Test):

    def test_empty_inputs(self):
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

Test.run_all()

In [ ]:
Test.run_all(only="match")